# State Space Search with A* Search Algorithm

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. 
The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

For the purposes of this exercise, we're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
🪨       hills      5
🐊       swamp      7
🗻       mountains  impassible
```

The raw format of the map will then look something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾🗻🗻🗻🌾🌾🌾
🌾🌾🗻🗻🌾🌾🌾
🌾🌾🗻🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The Maps

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [61]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾'],
['🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🐊', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌲', '🌲', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🪨', '🪨', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🌾', '🌾', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🪨', '🪨', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🪨', '🪨', '🗻', '🗻', '🌾', '🗻', '🗻', '🪨', '🪨', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾']
]

In [62]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. There are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well. 

When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in a *successor* function.

In [63]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [64]:
COSTS = { '🌾': 1, '🌲': 3, '🪨': 5, '🐊': 7}

In [65]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

<a id="heuristic"></a>
## heuristic

`heuristic` is a helper function that calculates h(n) in the A* search algorithm. The heuristic function estimates the remaining cost to get to the goal position. The heuristic function is necessary in A* search to guide the search algorithm towards the goal efficiently. A* search combines both the cost to reach a node from the start node (g(n)) and an estimate of the cost from the current node to the goal node (h(n)). The heuristic function provides this estimate. The key idea behind A* search is to prioritize exploring nodes that are likely to lead to the goal. The heuristic function helps in estimating the potential cost from a node to the goal, allowing the algorithm to make informed decisions about which nodes to explore next.


**Used by**: [a_star_search](#a_star_search)

* **position** tuple: the current position.
* **goal** tuple: the goal position.

**returns** int: the distance between the current position and the goal position.

In [2]:
def heuristic(position, goal): 
    x1, y1 = position
    x2, y2 = goal
    return abs(x2 - x1) + abs(y2 - y1)

In [3]:
# assertions/unit tests

# Test case 1: Positions at (0, 0) and (3, 4)
position = (0, 0)
goal = (3, 4)
expected_cost = 7  # Manhattan distance: |3 - 0| + |4 - 0|
assert heuristic(position, goal) == expected_cost

# Test case 2: Positions at (2, 2) and (2, 2)
position = (2, 2)
goal = (2, 2)
expected_cost = 0  # Manhattan distance: |2 - 2| + |2 - 2|
assert heuristic(position, goal) == expected_cost

# Test case 3: Positions at (5, 5) and (1, 1)
position = (5, 5)
goal = (1, 1)
expected_cost = 8  # Manhattan distance: |1 - 5| + |1 - 5|
assert heuristic(position, goal) == expected_cost

<a id="get_neighbors"></a>
## get_neighbors

`get_neighbors` takes a position and returns a list of valid neighboring positions. It uses the moves list to generate the neighbor positions by adding the offsets to the current position. The resulting neighbors are filtered to ensure they are within the bounds of the world grid. The function also specifies mountain spaces as impassible. These neighboring positions are important for expanding the search space and exploring possible paths towards the goal.


**Used by**: [a_star_search](#a_star_search)

* **position** tuple: a tuple of the agent's current position.
* **world** list: the list of strings representing the world to be explored.
* **moves** list: the list of allowable moves(up/down/left/right).


**returns** list: a list of neighboring positions

In [4]:
def get_neighbors(position, world, moves):
    x, y = position
    neighbors = [(x + dx, y + dy) for dx, dy in moves]
    neighbors = [(nx, ny) for nx, ny in neighbors if 0 <= nx < len(world) and 0 <= ny < len(world[0]) and world[nx][ny] != '🗻']
    return neighbors

In [10]:
# assertions/unit tests

# Test case 1: Positions at (1, 1) with no obstacles
position = (1, 1)
world = [['', '', ''],
            ['', '', ''],
            ['', '', '']]
moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
expected_neighbors = [(1, 2), (1, 0), (2, 1), (0, 1)]
assert get_neighbors(position, world, moves) == expected_neighbors

# Test case 2: Positions at (0, 0) with obstacles and boundary checks
position = (0, 0)
world = [['🗻', '', ''],
            ['', '', ''],
            ['', '', '']]
moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
expected_neighbors = [(0, 1), (1, 0)]
assert get_neighbors(position, world, moves) == expected_neighbors

<a id="calculate_costs"></a>
## calculate_costs

`calculate_costs` calculates the cost of moving from position to neighbor. This function is necessary in A* search to determine the cost of reaching a neighbor position from a given position. It combines the costs incurred from the parent position, the terrain type of the neighbor position, and the movement cost.


**Used by**: [a_star_search](#a_star_search)

* **parent_cost** int: the value of the parent cost.
* **position** tuple: a tuple of the agent's current position.
* **neighbor** tuple: a tuple of the neighboring position.
* **world** list: the list of strings representing the world to be explored.
* **costs** dict: a dictionary that maps cost values to terrain values.

**returns** int: the total cost value.

In [11]:
def calculate_cost(parent_cost, position, neighbor, world, costs):
    x1, y1 = position
    x2, y2 = neighbor
    terrain_cost = costs[world[x2][y2]]
    move_cost = abs(x2 - x1) + abs(y2 - y1)
    return parent_cost + terrain_cost + move_cost

In [16]:
# assertions/unit tests

# Test case 1: Parent cost is 0, no terrain cost, and Manhattan distance of 1
parent_cost = 0
position = (1, 1)
neighbor = (2, 1)
world = [['', '', ''],
            ['', '', ''],
            ['', '', '']]
costs = {'': 0}
expected_cost = 1
assert calculate_cost(parent_cost, position, neighbor, world, costs) == expected_cost

# Test case 2: Parent cost is 5, terrain cost of 3, and Manhattan distance of 2
parent_cost = 5
position = (0, 0)
neighbor = (1, 2)
world = [['', '', '🌲'],
            ['', '', ''],
            ['', '', '']]
costs = {'': 0, '🌲': 3}
expected_cost = 8
assert calculate_cost(parent_cost, position, neighbor, world, costs) == expected_cost

# Test case 3: Parent cost is 2, no terrain cost, and Manhattan distance of 0
parent_cost = 2
position = (3, 4)
neighbor = (3, 4)
world = [['', '', '', '', '', ''],
            ['', '', '', '', '', ''],
            ['', '', '', '', '', ''],
            ['', '', '', '', '', ''],
            ['', '', '', '', '', '']]
costs = {'': 0}
expected_cost = 2
assert calculate_cost(parent_cost, position, neighbor, world, costs) == expected_cost


<a id="a_star_search"></a>
## a_star_search

`a_start_search` is a function for the A* search algorithm 

This implementation uses a priority queue (frontier) to store the open nodes and is sorted based on the sum of the cost to reach the node and the estimated cost to the goal (f). A* search iteratively selects the node with the minimum f value from the open list and expands its neighbors. The costs are calculated based on the terrain type and the movement model, and the heuristic function is used to estimate the remaining cost to the goal.


**Uses**: [heuristic](#heuristic), [get_neighbors](#get_neighbors), [calculate_costs](#calculate_costs)

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [72]:
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    
    #initilize data structures
    frontier = [(start, 0)]  
    explored = set()
    g = {start: 0}
    f = {start: heuristic(start, goal)}
    parents = {}
    
    while frontier:
        current, current_cost = min(frontier, key=lambda x: x[1] + f[x[0]])
        
        if current == goal:
            path = []
            while current in parents:
                path.insert(0, current)
                current = parents[current]
            path.insert(0, start)
            return path
        
        frontier.remove((current, current_cost))
        explored.add(current)
        
        for neighbor in get_neighbors(current, world, moves):
            if neighbor in explored:
                continue
            
            cost = calculate_cost(current_cost, current, neighbor, world, costs)
            
            if neighbor not in frontier or cost < g[neighbor]:
                g[neighbor] = cost
                f[neighbor] = cost + heuristic(neighbor, goal)
                parents[neighbor] = current
                
                if neighbor not in frontier:
                    frontier.append((neighbor, cost))
    return [] 

## pretty_print_path

`pretty_print_path` is a function for the A* search algorithm 
What is it?
What does it do?

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - the path cost.

In [73]:
def pretty_print_path( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    
    path_cost = 0
    
    for i in range(len(world)):
        for j in range(len(world[0])):
            position = (i, j)
            
            if position == goal:
                print("🎁", end=" ")
            elif position in path:
                if position[0] < path[path.index(position) + 1][0]:
                    print("⏬", end=" ")  # Move down
                elif position[0] > path[path.index(position) + 1][0]:
                    print("⏪", end=" ")  # Move left
                elif position[1] < path[path.index(position) + 1][1]:
                    print("⏩", end=" ")  # Move right
                elif position[1] > path[path.index(position) + 1][1]:
                    print("⏫", end=" ")  # Move up
                path_cost += costs[world[i][j]]
            else:
                print(world[i][j], end=" ")
        
        print()  # Move to the next row
    
    return path_cost

## Navigating Small World

In [74]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏩ ⏩ ⏩ ⏩ ⏩ ⏩ ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 🎁 
total path cost: 12
[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6), (6, 6)]


## Navigating Big World

In [75]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬ 🌾 🌾 🌾 🌾 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 
⏬ 🌾 🌾 🌾 🌾 🌾 🌾 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🗻 🗻 🗻 🗻 🗻 🗻 🗻 🌾 🌾 
⏬ 🌾 🌾 🌾 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🪨 🪨 🪨 🗻 🗻 🪨 🪨 
⏬ 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🌲 🌲 🌲 🌲 🐊 🐊 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🪨 🪨 🗻 🗻 🪨 🌾 
⏬ 🌾 🌾 🪨 🪨 🗻 🗻 🌲 🌲 🌾 🌾 🐊 🐊 🐊 🐊 🌲 🌲 🌲 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🪨 🌾 
⏬ 🪨 🪨 🪨 🗻 🗻 🪨 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🐊 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🪨 🌾 🌾 
⏬ 🪨 🪨 🗻 🗻 🪨 🪨 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🐊 🐊 🐊 🌾 🌾 🌾 🌾 🌾 🪨 🌾 🌾 🌾 
⏬ 🌾 🪨 🪨 🪨 🪨 🪨 🌾 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🌾 🌾 
⏬ 🌾 🌾 🪨 🪨 🪨 🌾 🌾 🌾 🌾 🌾 🌾 🪨 🪨 🗻 🗻 🌾 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🌾 🌾 🌾 
⏬ 🌾 🌾 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🗻 🗻 🗻 🗻 🌾 🌾 🌾 🐊 🌾 🪨 🪨 🪨 🌾 🌾 🌾 🌾 
⏬ 🌾 🐊 🐊 🐊 🐊 🐊 🌾 🪨 🪨 🗻 🗻 🗻 🪨 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🪨 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🌾 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🗻 🗻 🪨 🌾 🐊 🐊 🐊 🐊 🌾 🌾 🌾 🪨 🗻 🪨 🌾 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🌲 🌲 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🌾 🌾 🌾 🌾 🌾 
⏩ ⏬ 🌾 🌾 🗻 🌾 🌾 🌲 🌲 🌲 🌲 🪨 🪨 🪨 🪨 🌾 🐊 🐊 🐊 🌾 🌾 🪨 🗻 🪨 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🪨 🪨 🌾 🐊 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 
🌾 ⏬ 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🌾 🌾 🗻 🗻 🗻 🌾 🌾 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🗻 🌾 🌾 🌾 🌾 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 